# Muscle segmentation L3 volume

In [1]:
import torch 
from Unet3plus.unet3plus import UNet_3Plus_3D

model_3D  = UNet_3Plus_3D(n_classes=1, in_channels=1, feature_scale=4)
model_3D.load_state_dict(torch.load('models/best_unet3plusmodel.pth', weights_only=True, map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
import pickle
import torch

from monai.transforms import (
    EnsureChannelFirstd,
    Compose,
    ScaleIntensityd,
    Resized,
    AsDiscrete
)
from monai.data import Dataset, DataLoader

# load dataset
with open('datasetpkl/datasetpkl3d/3d_dataset_test.pkl', 'rb') as file:
    test_data = pickle.load(file)

test_data_dict = [
    {"img": img[:19], "seg": ann[:19]}
    for img, ann in zip(test_data['images'], test_data['annotations'])
]

# define transformation
transforms = Compose(
    [
        ScaleIntensityd(keys=["img"]),
        EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),
        Resized(keys=['img'], spatial_size=(19, 256, 256), mode='area', anti_aliasing=True),
        Resized(keys=['seg'], spatial_size=(19, 256, 256), mode='nearest')
    ]
)

post_trans = Compose([AsDiscrete(threshold=0.5)])

test_ds = Dataset(data=test_data_dict, transform=transforms)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)